# ☁️ Lab 6: Azure AI Foundry
## Module 6 - Azure-Native Agent Architecture

**Duration:** 20 minutes

**Objectives:**
- Understand Azure AI Foundry architecture
- Configure Managed Identity authentication
- Build a loan advisor with RAG pattern

In [ ]:
!pip install openai azure-identity -q
print("✅ Packages installed")

In [ ]:
import os, json

DEMO_MODE = False
client = None
MODEL_NAME = "gpt-4o"

try:
    from google.colab import userdata
    AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
    AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
    try: MODEL_NAME = userdata.get('AZURE_OPENAI_DEPLOYMENT')
    except: pass
    if AZURE_OPENAI_KEY and AZURE_OPENAI_ENDPOINT:
        if not AZURE_OPENAI_ENDPOINT.startswith('http'):
            AZURE_OPENAI_ENDPOINT = 'https://' + AZURE_OPENAI_ENDPOINT
        print(f"✅ Credentials loaded. Model: {MODEL_NAME}")
    else: raise ValueError()
except: print("⚠️ DEMO MODE"); DEMO_MODE = True

if not DEMO_MODE:
    from openai import AzureOpenAI
    client = AzureOpenAI(api_key=AZURE_OPENAI_KEY, api_version="2024-06-01", azure_endpoint=AZURE_OPENAI_ENDPOINT)
    print("✅ Client ready")

## Azure AI Foundry Architecture

```
┌─────────────────────────────────────────┐
│         AZURE AI FOUNDRY                │
│  Agent Service │ Prompt Flow │ Evals   │
├─────────────────────────────────────────┤
│         Azure OpenAI Service            │
│   GPT-4o │ GPT-4o-mini │ Embeddings    │
└─────────────────────────────────────────┘
                    │
┌─────────────────────────────────────────┐
│  AI Search │ Cosmos DB │ Functions     │
└─────────────────────────────────────────┘
```

## Part 1: Loan Advisor Flow

In [ ]:
class LoanAdvisorFlow:
    def __init__(self, client):
        self.client = client
    
    def retrieve_context(self, query):
        """Simulate Azure AI Search"""
        return json.dumps({
            "mortgage": {"min_credit": 620, "max_dti": 0.43, "rates": "6.5-7.5%"},
            "personal": {"min_credit": 580, "max_amount": 50000, "rates": "8.99-24.99%"}
        })
    
    def get_customer_profile(self, customer_id):
        return json.dumps({"credit_score": 720, "income": 85000, "tier": "Gold"})
    
    def generate_response(self, query, context, profile):
        if DEMO_MODE or not self.client:
            return f"Based on your Gold tier status and 720 credit score, you qualify for our best mortgage rates (6.5%). [DEMO]"
        
        response = self.client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role": "system", "content": f"Loan advisor. Policies: {context}. Customer: {profile}"},
                      {"role": "user", "content": query}]
        )
        return response.choices[0].message.content
    
    def run(self, query, customer_id):
        print("\n🔄 Loan Advisor Flow")
        print("📚 Step 1: Retrieving policies...")
        context = self.retrieve_context(query)
        print("👤 Step 2: Getting profile...")
        profile = self.get_customer_profile(customer_id)
        print("🤖 Step 3: Generating response...")
        return self.generate_response(query, context, profile)

flow = LoanAdvisorFlow(client)
result = flow.run("What mortgage options do I qualify for?", "C-12345")
print(f"\n📋 Response: {result}")

## Part 2: Managed Identity (Production)

```python
# In production Azure - no API keys needed!
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
client = AzureOpenAI(azure_ad_token_provider=token_provider, ...)
```

## ✅ Lab 6 Complete!

**Key Takeaways:**
- Azure AI Foundry provides managed agent hosting
- Managed Identity eliminates secret management
- Prompt Flow enables visual orchestration

**Next:** `07_tool_orchestration.ipynb`